In [2]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import json
import re

# Démarrage du chronomètre pour calculer la durée d'exécution du script
start_time = time.time()

# Définition de la variable initial pour pour suivre la progression
pourcent = 71

# Définition de la plage d'ID dans l'url d'Allociné
depart_id = 200000
total_id = 400000

# Définition de la fonction pour scraper les données d'Allociné
def allocine_scrap(i):
    global pourcent
    # Initialisation des variables pour les informations à récupérer
    nom = recompenses = carriere_duree = films_series = naissance = pays = metier_1 = metier_2 = metier_3 = None
    
    try:
        # Construction de l'URL pour chaque film basé sur son ID
        url = f'https://www.allocine.fr/personne/fichepersonne_gen_cpersonne={i}.html'
        res = session.get(url, allow_redirects=True)

        # Print du % avec une intervalle de 24 tests
        pourcent += 1
        if pourcent == 72:
            print('Execution :', round(((i-depart_id)/(total_id-depart_id))*100, 2), '%')
            pourcent = 0
        
        # Vérification du statut de la réponse, retourne des "0" si le code HTTP n'est pas 200 ou si l'URL a été redirigée
        if res.status_code != 200:
            return i, "0", "0", "0", "0", "0", "0", "0", "0", "0"
        if res.url != url:
            return i, "0", "0", "0", "0", "0", "0", "0", "0", "0"
        
        # Utilisation de BeautifulSoup pour parser le contenu HTML
        soup = BeautifulSoup(res.content, 'lxml')

        # Extraction du nom de la personne.
        nom = soup.find('div', class_='titlebar-title titlebar-title-xl')

        # Extraction du premier bloc d'info (metiers, pays, date de naissance)
        infos = soup.find_all('div', class_="meta-body-item")
        for info in infos:
            info_next = info.find('span', class_='light')
            if info_next:
                if info_next.text.strip() == 'Métiers' or info_next.text.strip() == 'Métier':
                    metiers = info.find_all('span', class_=lambda x: x and 'dark-grey-link' in x)
                    if metiers:
                        if len(metiers) > 0 and metiers[0]:
                            metier_1 = metiers[0]
                        if len(metiers) > 1 and metiers[1]:
                            metier_2 = metiers[1]
                        if len(metiers) > 2 and metiers[2]:
                            metier_3 = metiers[2]
                    else:
                        metier_1 = info_next.find_next('strong')
                if info_next.text.strip() == 'Nationalité':
                    pays = info_next.find_next('div', class_='dark-grey')
                if info_next.text.strip() == 'Naissance':
                    naissance = info_next.find_next('span', class_=lambda x: x and 'dark-grey-link' in x)
        
        # Extraction du deuxième bloc d'info (durée de la carrière, nombre de films et series, nombre de recompenses)
        stats_numbers = soup.find_all(class_="stats-number")
        for stats_number in stats_numbers:
            stats_info = stats_number.find_next(class_="stats-info").get_text(strip=True)
            if "ans de carrière" in stats_info:
                carriere_duree = stats_number.get_text(strip=True)
            if "an de carrière" in stats_info:
                carriere_duree = stats_number.get_text(strip=True)
            elif "films et séries tournés" in stats_info:
                films_series = stats_number.get_text(strip=True)
            elif "film et série tourné" in stats_info:
                films_series = stats_number.get_text(strip=True)
            elif "prix" in stats_info:
                recompenses = stats_number.get_text(strip=True)
            elif "nominations" in stats_info:
                recompenses = 0
            elif "nomination" in stats_info:
                recompenses = 0
        
    # Gestion des exceptions liées aux requêtes HTTP
    except requests.exceptions.RequestException:
        return i, "0", "0", "0", "0", "0", "0", "0", "0", "0"
    
    # Retourne les données extraites ou des "0" si les données ne sont pas disponibles
    resultat = (
        i,
        nom.text.strip() if nom else "0",
        recompenses if recompenses else "0",
        carriere_duree if carriere_duree else "0",
        films_series if films_series else "0",
        naissance.text.strip() if naissance else "0",
        pays.text.strip() if pays else "0",
        metier_1.text.strip() if metier_1 else "0",
        metier_2.text.strip() if metier_2 else "0",
        metier_3.text.strip() if metier_3 else "0",
    )
    return resultat

# Initialisation d'un DataFrame pour stocker les données extraites   
df = pd.DataFrame(columns=['ID', 'nom', 'recompenses', 'carriere_duree', 'films_series', 'naissance', 'pays', 'metier_1', 'metier_2', 'metier_3']) 
session = requests.Session()

# Utilisation d'un ThreadPoolExecutor pour exécuter les scrapings en parallèle et utiliser les differents coeurs logique du PC.
with ThreadPoolExecutor(max_workers=6) as executor:

    # Soumission des tâches de scraping pour chaque ID de film entre 1 et la variable total_film
    tests = [executor.submit(allocine_scrap, i) for i in range(depart_id, total_id)]
    for test in tests:
        i, nom, recompenses, carriere_duree, films_series, naissance, pays, metier_1, metier_2, metier_3 = test.result()
        variables = (nom, recompenses, carriere_duree, films_series, naissance, pays, metier_1, metier_2, metier_3)
        if any(v != "0" for v in variables):
            df.loc[i] = [i, nom, recompenses, carriere_duree, films_series, naissance, pays, metier_1, metier_2, metier_3]
            #display(df.loc[i])

# Arrêt du chronomètre et affichage de la durée d'exécution
end_time = time.time()
execution_time = end_time - start_time
print(f"Le script a pris {execution_time} secondes pour s'exécuter.")

# Sauvegarde des résultats dans un fichier CSV
df.to_csv(f'allocine_{depart_id}_{total_id}.csv', index=False)

# Affichage du DataFrame final
display(df)

Execution : 0.0 %
Execution : 0.04 %
Execution : 0.07 %
Execution : 0.11 %
Execution : 0.14 %
Execution : 0.18 %
Execution : 0.22 %
Execution : 0.25 %
Execution : 0.29 %
Execution : 0.33 %
Execution : 0.36 %
Execution : 0.4 %
Execution : 0.43 %
Execution : 0.47 %
Execution : 0.5 %
Execution : 0.54 %
Execution : 0.58 %
Execution : 0.61 %
Execution : 0.65 %
Execution : 0.68 %
Execution : 0.72 %
Execution : 0.76 %
Execution : 0.79 %
Execution : 0.83 %
Execution : 0.86 %
Execution : 0.9 %
Execution : 0.94 %
Execution : 0.97 %
Execution : 1.01 %
Execution : 1.04 %
Execution : 1.08 %
Execution : 1.12 %
Execution : 1.15 %
Execution : 1.19 %
Execution : 1.22 %
Execution : 1.26 %
Execution : 1.3 %
Execution : 1.33 %
Execution : 1.37 %
Execution : 1.4 %
Execution : 1.44 %
Execution : 1.48 %
Execution : 1.51 %
Execution : 1.55 %
Execution : 1.58 %
Execution : 1.62 %
Execution : 1.66 %
Execution : 1.69 %
Execution : 1.73 %
Execution : 1.76 %
Execution : 1.8 %
Execution : 1.84 %
Execution : 1.87 %
